In [132]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import json

In [133]:
# Use dask to read in the csv file as a dataframe
listings_df = dd.read_csv("Resources/listings.csv", assume_missing = True)

In [134]:
# Select only the columns we want for our analysis
listings_df = listings_df[["id", "neighbourhood", "latitude", "longitude", "room_type", "price", "availability_365", "calculated_host_listings_count"]].copy()
listings_df.head()

,id,neighbourhood,latitude,longitude,room_type,price,availability_365,calculated_host_listings_count
0,360.0,Highland,39.766415,-105.002098,Entire home/apt,91.0,179.0,2.0
1,364.0,Five Points,39.766720,-104.979060,Entire home/apt,179.0,358.0,1.0
2,590.0,North Park Hill,39.755110,-104.911090,Private room,59.0,146.0,2.0
3,592.0,North Park Hill,39.754810,-104.911060,Private room,58.0,210.0,2.0
4,1940.0,Baker,39.720205,-104.997499,Entire home/apt,77.0,243.0,1.0


In [135]:
# Find datatypes for each column
print(listings_df.dtypes)

id                                        float64
neighbourhood                     string[pyarrow]
latitude                                  float64
longitude                                 float64
room_type                         string[pyarrow]
price                                     float64
availability_365                          float64
calculated_host_listings_count            float64
dtype: object


In [136]:
# Change datatypes
listings_df["id"] = listings_df["id"].astype(int)
listings_df["availability_365"] = listings_df["availability_365"].astype(int)
listings_df["calculated_host_listings_count"] = listings_df["calculated_host_listings_count"].astype(int)

# Rename columns
listings_df = listings_df.rename(columns={
    "id": "ID",
    "neighbourhood": "Neighborhood",
    "latitude": "Latitude",
    "longitude": "Longitude",
    "room_type": "Listing Type",
    "price": "Price",
    "availability_365": "Available Days per Year",
    "calculated_host_listings_count": "Host Listings Count"
})

In [137]:
# Check datatypes were successfully changed
print(listings_df.dtypes)

ID                                   int64
Neighborhood               string[pyarrow]
Latitude                           float64
Longitude                          float64
Listing Type               string[pyarrow]
Price                              float64
Available Days per Year              int64
Host Listings Count                  int64
dtype: object


In [138]:
# Find out if there are null values in each column
print(listings_df.isnull().sum().compute())

ID                           0
Neighborhood                 0
Latitude                     0
Longitude                    0
Listing Type                 0
Price                      573
Available Days per Year      0
Host Listings Count          0
dtype: int64


In [139]:
# Remove all rows with null values
listings_df = listings_df.dropna()

In [140]:
# Recheck for null values
print(listings_df.isnull().sum().compute())

ID                         0
Neighborhood               0
Latitude                   0
Longitude                  0
Listing Type               0
Price                      0
Available Days per Year    0
Host Listings Count        0
dtype: int64


In [141]:
# Get statistical information
summary_stats = listings_df[["Latitude", "Longitude", "Price", "Available Days per Year"]].describe()
print(summary_stats.compute())

          Latitude    Longitude        Price  Available Days per Year
count  4578.000000  4578.000000  4578.000000              4578.000000
mean     39.742149  -104.975373   162.639144               219.004806
std       0.031772     0.061382   239.225490               116.491189
min      39.625750  -105.102804    10.000000                 0.000000
25%      39.728498  -105.016473    84.000000               116.000000
50%      39.748135  -104.984050   117.000000               237.000000
75%      39.762350  -104.957370   178.000000               334.000000
max      39.847417  -104.673817  9999.000000               365.000000


In [142]:
# Find out how many listings have "Available Days per Year" as 0
print(listings_df.query("`Available Days per Year` == 0").count().compute())

ID                         77
Neighborhood               77
Latitude                   77
Longitude                  77
Listing Type               77
Price                      77
Available Days per Year    77
Host Listings Count        77
dtype: int64


In [143]:
# Filter out all rows where "Available Days per Year" is 0
listings_df = listings_df.query("`Available Days per Year` ! = 0")

In [144]:
# Look at statistical information again after filtering out rows
summary_stats = listings_df[["Latitude", "Longitude", "Price", "Available Days per Year"]].describe()
print(summary_stats.compute())

          Latitude    Longitude        Price  Available Days per Year
count  4501.000000  4501.000000  4501.000000              4501.000000
mean     39.742191  -104.975155   162.798267               222.751389
std       0.031843     0.061594   240.860574               113.875617
min      39.625750  -105.102804    10.000000                 1.000000
25%      39.728490  -105.016480    84.000000               121.000000
50%      39.748120  -104.983920   116.000000               240.000000
75%      39.762540  -104.956978   178.000000               335.000000
max      39.847417  -104.673817  9999.000000               365.000000


In [145]:
listings_df_cleaned = listings_df.copy()
listings_df_cleaned.head()

,ID,Neighborhood,Latitude,Longitude,Listing Type,Price,Available Days per Year,Host Listings Count
0,360,Highland,39.766415,-105.002098,Entire home/apt,91.0,179,2
1,364,Five Points,39.766720,-104.979060,Entire home/apt,179.0,358,1
2,590,North Park Hill,39.755110,-104.911090,Private room,59.0,146,2
3,592,North Park Hill,39.754810,-104.911060,Private room,58.0,210,2
4,1940,Baker,39.720205,-104.997499,Entire home/apt,77.0,243,1


In [146]:
# Export cleaned data to a new csv
listings_df_cleaned.to_csv('Resources/cleaned_listings.csv', index=False, single_file=True)

['/Users/angelinamurdock/Desktop/Classwork/Project-5_Group-3/Resources/cleaned_listings.csv']

In [148]:
# Convert cleaned_listings.csv into a .json file 
listings_df = dd.read_csv('Resources/cleaned_listings.csv')

# Compute the DataFrame to Pandas (if the dataset fits into memory)
listings_pandas_df = listings_df.compute()

# Convert to list of dictionaries (this is how the data will be represented in JavaScript)
data_as_dict = listings_pandas_df.to_dict(orient='records')

# Write the JavaScript data to a .json file
with open("Resources/listings.json", "w") as json_file:
    json.dump(data_as_dict, json_file, indent=4)